In [1]:
Threads.nthreads()

1

In [2]:
using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, POMDPTools, POMDPPolicies, ParticleFilters, DiscreteValueIteration

In [3]:
using Statistics, Plots, CSV, DataFrames, JLD2

In [4]:
using D3Trees, Random

In [5]:
include("temps/BasicPOMCP_2.jl")

┌ Warning: POMDPs.@implemented is deprecated, use POMDPLinter.@implemented instead.
└ @ POMDPs /home/pranay/.julia/packages/POMDPs/XBTe5/src/deprecated.jl:10


Main.BasicPOMCP

In [21]:
component_id = "5";
replacement_cost = 60.0;
inspection_cost = 1.0;
horizon = 100;

dynamics = []
open("dynamics/dynamics_"*component_id*".csv") do f
    line = 0  
    while ! eof(f) 
        # read a new / next line for every iteration          
        s = readline(f)
        s = parse.(Float64, split(chop(s; head=1, tail=1), ','))
        push!(dynamics,s)
        line += 1
    end
end

budget = 500.0
buildingprob = QuickPOMDP(
        actions = ["do-nothing", "inspect", "maintain"], 
        obstype = Tuple{Int64, Int64},
        discount = 1.0,

        transition = function(s,a)
            next_states_inspect = []
            next_states_nothing = []
            for i in 0:s[1]
                push!(next_states_inspect, (i,s[2]+inspection_cost))
                push!(next_states_nothing, (i,s[2]))
            end
            if a == "maintain"
                return Deterministic((100, s[2] + replacement_cost))
            elseif a == "inspect"
                return SparseCat(next_states_inspect, dynamics[s[1]+1])
            elseif a == "do-nothing"
                return SparseCat(next_states_nothing, dynamics[s[1]+1])
            end
        end,

        observation = function(s,a,sp)
            if a == "inspect"
                return Deterministic(sp)
            elseif a == "maintain"
                return Deterministic(sp)
            elseif a == "do-nothing"
                return Deterministic((101, sp[2]))
            end
        end,

        reward = function(s,a,sp)
            if sp[1] > 0
                return 1
            else
                return -1
            end
        end,
        isterminal = function(s)
            if s[2] + replacement_cost > budget
                return true
            else
                return false
            end
        end,
        initialstate = Deterministic((100, 0)))


solver_q = BasicPOMCP.POMCPSolver(max_depth=20, tree_queries=2000, c=10, tree_in_info = true);
planner_q = solve(solver_q, buildingprob);

a, info = action_info(planner_q, initialstate(buildingprob), tree_in_info=true)
inchrome(D3Tree(info[:tree], init_expand=2))

#for (s, a, o, r, b) in stepthrough(buildingprob, planner_q, "s,a,o,r,b", max_steps = horizon)

    # print state, action, observation, reward, and belief info clearly
    # println("State: ", s)
    # println("Action: ", a)
    # println("Observation: ", o)
    # println("Reward: ", r)
    # println("Belief: ", b.particles[1][2])
    # println("")



Opening in existing browser session.


Process(`google-chrome /tmp/jl_OiwXZv/tree.html`, ProcessExited(0))